In [1]:
import ee, datetime
import pandas as pd
import numpy as np
import folium
import geehydro
import datetime as dt
import IPython.display

In [2]:
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [3]:
Shahpura_Lake = folium.Map(location=[23.205476997945688, 77.42096687745122], zoom_start=10)


In [4]:
sentinel = ee.ImageCollection("COPERNICUS/S2_SR").\
               filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)).\
               filterDate("2015-01-01","2023-01-01")
Shahpura_Lake_AOI = ee.Geometry.Polygon([[[77.42117255516425,23.20909792227606],[77.42057174034491,23.20799353262676],[77.42040007896796,23.207520219982936],[77.4200996715583,23.2069285768207],[77.41949885673895,23.207125791499145],[77.41902678795233,23.207007462726995],[77.41889804191962,23.206258044736966],[77.41868346519843,23.20550862254438],[77.41868346519843,23.204995857570196],[77.41851180382147,23.204680308915954],[77.41816848106757,23.203970321719954],[77.41885512657538,23.20314199855786],[77.41928428001776,23.202787001345527],[77.4204429943122,23.201840337503217],[77.42082923241034,23.201406447668166],[77.42125838585272,23.20089366695645],[77.42173045463934,23.20101200114144],[77.4224600154914,23.20140644766817],[77.42327540703192,23.20152478139923],[77.42361872978583,23.20187978196384],[77.42366164513007,23.202313670262438],[77.42413371391669,23.202589780264603],[77.42464869804755,23.203209392314182],[77.42507785148993,23.203761608047508],[77.42524951286688,23.20451104003633],[77.42451995201483,23.20569434514694],[77.42387622185126,23.206207107441198],[77.42348998375311,23.20707485453413],[77.42306083031075,23.208179251775938],[77.42284625358954,23.208692004535596],[77.42207377739327,23.20853423466525],[77.42143004722968,23.20837646460864],[77.42117255516425,23.20909792227606]]])

sentinel_AOI = sentinel.filterBounds(Shahpura_Lake_AOI)
               

In [5]:
print('Total number of images :', sentinel_AOI.size().getInfo())

Total number of images : 166


In [6]:
def calculate_NDSI(image):
  ndsi = image.normalizedDifference(['B11', 'B12']).rename('NDSI')
  return image.addBands(ndsi)
ndsi = sentinel_AOI.map(calculate_NDSI)

In [7]:
def calculate_mean_NDSI(image):
  image = ee.Image(image)
  mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['NDSI']),
                           geometry = Shahpura_Lake_AOI,
                           scale = image.projection().nominalScale().getInfo(),
                           maxPixels = 100000,
                           bestEffort = True);
  return mean.get('NDSI').getInfo()

In [8]:
Images_ndsi = ndsi.select('NDSI').toList(ndsi.size())
ndsi_coll = []
for i in range(Images_ndsi.length().getInfo()):
  image = ee.Image(Images_ndsi.get(i-1))
  temp_ndsi = calculate_mean_NDSI(image)
  ndsi_coll.append(temp_ndsi)

In [9]:
dates = np.array(ndsi.aggregate_array("system:time_start").getInfo())
day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

In [10]:
df2 = pd.DataFrame(ndsi_coll, index = day, columns = ['Salinty'])
df2.index = pd.to_datetime(df2.index, format="%Y/%m/%d")
df2.sort_index(ascending = True, inplace = True)

In [11]:
df2

,Salinty
2018-12-18,0.039769
2018-12-23,0.166848
2018-12-28,0.029120
2019-01-02,0.077468
2019-01-07,0.058016
...,...
2022-12-02,0.016628
2022-12-07,0.010108
2022-12-17,0.013112
2022-12-22,0.017932


In [12]:
df2.to_csv('Salinty.csv')

In [13]:
sentinel3 = ee.ImageCollection("COPERNICUS/S3/OLCI").\
              filterDate("2020-01-01","2020-02-02")
sarapangi_AOI = ee.Geometry.Polygon([[[77.42117255516425,23.20909792227606],[77.42057174034491,23.20799353262676],[77.42040007896796,23.207520219982936],[77.4200996715583,23.2069285768207],[77.41949885673895,23.207125791499145],[77.41902678795233,23.207007462726995],[77.41889804191962,23.206258044736966],[77.41868346519843,23.20550862254438],[77.41868346519843,23.204995857570196],[77.41851180382147,23.204680308915954],[77.41816848106757,23.203970321719954],[77.41885512657538,23.20314199855786],[77.41928428001776,23.202787001345527],[77.4204429943122,23.201840337503217],[77.42082923241034,23.201406447668166],[77.42125838585272,23.20089366695645],[77.42173045463934,23.20101200114144],[77.4224600154914,23.20140644766817],[77.42327540703192,23.20152478139923],[77.42361872978583,23.20187978196384],[77.42366164513007,23.202313670262438],[77.42413371391669,23.202589780264603],[77.42464869804755,23.203209392314182],[77.42507785148993,23.203761608047508],[77.42524951286688,23.20451104003633],[77.42451995201483,23.20569434514694],[77.42387622185126,23.206207107441198],[77.42348998375311,23.20707485453413],[77.42306083031075,23.208179251775938],[77.42284625358954,23.208692004535596],[77.42207377739327,23.20853423466525],[77.42143004722968,23.20837646460864],[77.42117255516425,23.20909792227606]]])
sentinel3_AOI = sentinel3.filterBounds(sarapangi_AOI)

In [14]:
def calculate_SM(image):
  rgb = image.select(['Oa08_radiance', 'Oa06_radiance', 'Oa04_radiance'])\
              .multiply(ee.Image([0.00876539, 0.0123538, 0.0115198]))
  suspended_matter = rgb.select('Oa08_radiance').divide(rgb.select('Oa06_radiance')).rename('suspended_matter')
  return image.addBands(suspended_matter)
sm = sentinel3_AOI.map(calculate_SM)

In [15]:
def calculate_mean_SM(image):
  image = ee.Image(image)
  mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['suspended_matter']),
                           geometry = Shahpura_Lake_AOI,
                           scale = image.projection().nominalScale().getInfo(),
                           maxPixels = 100000,
                           bestEffort = True);
  return mean.get('suspended_matter').getInfo()

In [16]:
Images_sm = sm.select('suspended_matter').toList(sm.size())
sm_coll= []
for i in range(Images_sm.length().getInfo()):
  image = ee.Image(Images_sm.get(i-1))
  temp_sm = calculate_mean_SM(image)
  sm_coll.append(temp_sm)

In [17]:
dates = np.array(sm.aggregate_array("system:time_start").getInfo())
day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

In [18]:
df7 = pd.DataFrame(sm_coll, index = day, columns = ['Suspended Matter'])
df7.index = pd.to_datetime(df7.index, format="%Y/%m/%d")
df7.sort_index(ascending = True, inplace = True)

In [19]:
df7

,Suspended Matter
2020-01-01,0.538985
2020-01-01,0.497685
2020-01-02,0.642722
2020-01-04,0.625358
2020-01-05,0.637351
2020-01-05,0.639032
2020-01-06,0.487949
2020-01-08,0.518998
2020-01-09,0.515529
2020-01-09,0.419313
